In [ ]:
## BRANCH ADOPTION.py

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/sharb24/SBB/thural2/record.py').read())


In [ ]:
###################################################################################################
############################### BRANCH ADOPTION ###################################################

## this only pulls weekly data
dfbr=spark.table('anp_cabbtdct1_final.SBB_BRANCH_ADOPTION') 
df_br=dfbr.toPandas()

df_br1=df_br[(df_br.record_date>=fy_start) & (df_br.record_date<=rec_date)]

## add a new record_date variable
EDPP5=df_br1.copy()

EDPP5['month_dt'] = pd.to_datetime(EDPP5.prdate, format='%Y-%m-%d')
EDPP5['month_dt'] = EDPP5['month_dt'].dt.month

###### EXTRACT THE BBS AND RETAIL
EDPP5=EDPP5[(EDPP5.jobcode=='BBS') | (EDPP5.jobcode=='Retail')]

EDPP5.rename(columns = {"booking_transit":'branch'}, inplace = True)
EDPP5['branch']=EDPP5['branch'].str.strip()


In [ ]:
### merge to alignment
df3=spark.sql("select branch, am_cost_center, am_cost_center_name, sm_cost_center, sm_cost_center_name, \
    mon, case when AM_Cost_Center='3838' then '3838 - RAMSB' when AM_Cost_Center='5311' then '5311 - RAMSB' \
    else concat(AM_Cost_Center,' - ',AM_Cost_Center_Name) end as am_cost_center_full_name \
    from \
    (select branch,AM_Cost_Center, AM_Cost_Center_Name,SM_Cost_Center,SM_Cost_Center_Name, \
    record_date,cast(SUBSTR(record_date,5,2) as int) as mon, \
    row_number() over(partition by branch,cast(SUBSTR(record_date,5,2) as int) order by record_date desc) as ran \
    from anp_cabbtdct1_final.sbb_alignment \
    )n where ran=1")
df2=df3.withColumn("branch_0000",format_string("%04d","branch"))
dfsbb=df2.toPandas()

adopt00=pd.merge(EDPP5,dfsbb, left_on=['branch','month_dt'],right_on=['branch_0000','mon'],how="left")

import calendar
adopt00['month_dt'] = adopt00['month_dt'].apply(lambda x: calendar.month_name[x]).str.lower()

##rename branch
adopt00.rename(columns = {"branch_0000":'branch'}, inplace = True)

### group by job_code (BBS or Retail) and sum up by distinct trxn id
    ##keep only relevant variables
adopt1=adopt00[['transaction_id','jobcode','branch','am_cost_center', 'am_cost_center_name','am_cost_center_full_name',
              'sm_cost_center','sm_cost_center_name','month_dt']].drop_duplicates()
    
adopt2=adopt1.groupby(by=['jobcode','am_cost_center', 'am_cost_center_name','am_cost_center_full_name',
              'sm_cost_center','sm_cost_center_name','month_dt'],as_index=False).aggregate({
                    'transaction_id': 'nunique'
                     })


In [ ]:
## separate BBS from Retail and then divide to get branch adoption rate
##BBS
bbs_adopt=adopt2[(adopt2.jobcode=='BBS')]
bbs_adopt.rename(columns = {"jobcode":'jobcode_BBS', "transaction_id":'bbs_count'}, inplace = True)

##Retail
retail_adopt=adopt2[(adopt2.jobcode=='Retail')]
retail_adopt.rename(columns = {"jobcode":'jobcode_Retail', "transaction_id":'retail_count'}, inplace = True)

## remove un-wanted vars from Retail
retail_adopt.drop(columns=['sm_cost_center', 'sm_cost_center_name','am_cost_center_full_name','am_cost_center_name'])

## merge the two
adopt_full=pd.merge(bbs_adopt,retail_adopt, left_on=['month_dt','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                        'sm_cost_center','sm_cost_center_name'],
                    right_on=['month_dt','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                        'sm_cost_center','sm_cost_center_name'],how="outer")


In [ ]:
# fill in with 0's
adopt_full['bbs_count']=adopt_full['bbs_count'].fillna(0)
adopt_full['retail_count']=adopt_full['retail_count'].fillna(0)

## keep relevant vars
adopt_full1=adopt_full[['am_cost_center','am_cost_center_name','am_cost_center_full_name',
                        'sm_cost_center','sm_cost_center_name','jobcode_Retail','retail_count',
                        'jobcode_BBS','bbs_count', 'month_dt']]


In [ ]:
###############################################################################################################
##################################### CALCULATE BRANCH ADOPTION RATE ##########################################

adopt_full1['Branch Adoption percent']=(adopt_full1.bbs_count/(adopt_full1.bbs_count+adopt_full1.retail_count))

## include the metric name and scorecard date
adopt_full1['metric_name']='Branch Adoption percent'

#### transpose data
a =adopt_full1.pivot_table(index=['am_cost_center','am_cost_center_name','am_cost_center_full_name',
                        'sm_cost_center','sm_cost_center_name','metric_name'], 
                    columns=['month_dt'],
                     values='Branch Adoption percent', aggfunc='first', fill_value=0).reset_index()

## create a copy
adopt_full3=adopt_full1.copy()
## create a metric name
adopt_full3['metric_name']='Branch Adoption new accounts'

b =adopt_full3.pivot_table(index=['am_cost_center','am_cost_center_name','am_cost_center_full_name',
                        'sm_cost_center','sm_cost_center_name','metric_name'], 
                    columns=['month_dt'],
                     values='bbs_count', aggfunc='first', fill_value=0).reset_index()

##### append the two dataframes (2 metrics)
adopt_all=pd.concat([a,b],axis=0)


In [ ]:
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in adopt_all]

adopt_all.reset_index(drop=False, inplace=True)
adopt_new2= pd.concat([net_a[complementary], adopt_all], axis=1, join='outer')

##delete the extra filler row
adopt_new2= adopt_new2.drop(labels='a', axis=0)
##convert to float
adopt_new2=adopt_new2.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

adopt_new2['q1'] = adopt_new2[['november','december','january']].mean(axis=1)
adopt_new2['q2'] = adopt_new2[['february','march','april']].mean(axis=1)
adopt_new2['q3'] = adopt_new2[['may','june','july']].mean(axis=1)
adopt_new2['q4'] = adopt_new2[['august','september','october']].mean(axis=1)
adopt_new2['ytd'] = adopt_new2[['november','december','january','february','march','april','may','june','july','august','september','october']].mean(axis=1)


In [ ]:
## replace 0 with NaN so median only counts non-zeros
adopt_new2=adopt_new2.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_am_medianq1=adopt_new2.groupby(['metric_name'], as_index=False)['q1'].median()
df_am_medianq2=adopt_new2.groupby(['metric_name'], as_index=False)['q2'].median()
df_am_medianq3=adopt_new2.groupby(['metric_name'], as_index=False)['q3'].median()
df_am_medianq4=adopt_new2.groupby(['metric_name'], as_index=False)['q4'].median()
df_am_medianytd=adopt_new2.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_am_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_am_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_am_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_am_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_am_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
adopta=pd.merge(pd.merge(adopt_new2,df_am_medianq1,
                on=['metric_name'],
                how='left'),df_am_medianq2,on=['metric_name'],how='left')

adoptb=pd.merge(pd.merge(adopta,df_am_medianq3,
                on=['metric_name'],
                how='left'),df_am_medianq4,on=['metric_name'],how='left')

adoptc=pd.merge(adoptb,df_am_medianytd,on=['metric_name'],how="left")

adoptc['level']='AM'
adoptc['scorecard_filter']='SBB'
adoptc['employee_name']=adoptc.am_cost_center_full_name
adoptc['acf2_id']=''
adoptc['record_date']=rec_date

### get AM
adopt_am=adoptc[['metric_name','level','acf2_id','employee_name','november','december','january',
        'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


## create a copy for Total
adopt_am1=adopt_am.copy()
adopt_am1['scorecard_filter']='Total'


In [ ]:
## GROUP BY REGION_ID AND SUM UP
#### transpose data
a =adopt_full1.pivot_table(index=['sm_cost_center','sm_cost_center_name','metric_name'], 
                    columns=['month_dt'],
                     values='Branch Adoption percent', aggfunc='first', fill_value=0).reset_index()

## create a copy
adopt_full3=adopt_full1.copy()

## create a metric name
adopt_full3['metric_name']='Branch Adoption new accounts'

b =adopt_full3.pivot_table(index=['sm_cost_center','sm_cost_center_name','metric_name'], 
                    columns=['month_dt'],
                     values='bbs_count', aggfunc='first', fill_value=0).reset_index()

##### append the two dataframes (2 metrics)
adopt_all=pd.concat([a,b],axis=0)

cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in adopt_all]

adopt_all.reset_index(drop=False, inplace=True)
sm_ad= pd.concat([net_a[complementary], adopt_all], axis=1, join='outer',sort=False)

##delete the extra filler row
sm_ad= sm_ad.drop(labels='a', axis=0)
sm_ad= sm_ad.drop(columns='index', axis=0)

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in sm_ad]
sm_ad2= pd.concat([net_a[complementary], sm_ad], axis=1, join='outer',sort=False)


In [ ]:
##################################### ROLL UP TO SM ##########################################

## GROUP BY REGION_ID AND SUM UP
#### transpose data
a =adopt_full1.pivot_table(index=['sm_cost_center','sm_cost_center_name','metric_name'], 
                    columns=['month_dt'],
                     values='Branch Adoption percent', aggfunc='first', fill_value=0).reset_index()

## create a copy
adopt_full3=adopt_full1.copy()

## create a metric name
adopt_full3['metric_name']='Branch Adoption new accounts'

b =adopt_full3.pivot_table(index=['sm_cost_center','sm_cost_center_name','metric_name'], 
                    columns=['month_dt'],
                     values='bbs_count', aggfunc='first', fill_value=0).reset_index()

##### append the two dataframes (2 metrics)
adopt_all=pd.concat([a,b],axis=0)

cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in adopt_all]

adopt_all.reset_index(drop=False, inplace=True)
sm_ad= pd.concat([net_a[complementary], adopt_all], axis=1, join='outer',sort=False)

##delete the extra filler row
sm_ad= sm_ad.drop(labels='a', axis=0)

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in sm_ad]
sm_ad2= pd.concat([net_a[complementary], sm_ad], axis=1, join='outer',sort=False)

##delete the extra filler row
sm_ad2= sm_ad2.drop(labels='a', axis=0)
##convert to float
sm_ad2=sm_ad2.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

sm_ad2['q1'] = sm_ad2[['november','december','january']].mean(axis=1)
sm_ad2['q2'] = sm_ad2[['february','march','april']].mean(axis=1)
sm_ad2['q3'] = sm_ad2[['may','june','july']].mean(axis=1)
sm_ad2['q4'] = sm_ad2[['august','september','october']].mean(axis=1)
sm_ad2['ytd'] = sm_ad2[['november','december','january','february','march','april','may','june','july','august','september','october']].mean(axis=1)

## replace 0 with NaN so median only counts non-zeros
sm_ad2=sm_ad2.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_sm_medianq1=sm_ad2.groupby(['metric_name'], as_index=False)['q1'].median()
df_sm_medianq2=sm_ad2.groupby(['metric_name'], as_index=False)['q2'].median()
df_sm_medianq3=sm_ad2.groupby(['metric_name'], as_index=False)['q3'].median()
df_sm_medianq4=sm_ad2.groupby(['metric_name'], as_index=False)['q4'].median()
df_sm_medianytd=sm_ad2.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_sm_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_sm_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_sm_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_sm_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_sm_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
sm_ad3=pd.merge(pd.merge(sm_ad2,df_sm_medianq1,
                on=['metric_name'],
                how='left'),df_sm_medianq2,on=['metric_name'],how='left')

sm_ad4=pd.merge(pd.merge(sm_ad3,df_sm_medianq3,
                on=['metric_name'],
                how='left'),df_sm_medianq4,on=['metric_name'],how='left')

sm_ad5=pd.merge(sm_ad4,df_sm_medianytd,on=['metric_name'],how="left")

sm_ad5['acf2_id']=''
sm_ad5['level']='SM'
sm_ad5['scorecard_filter']='SBB'
sm_ad5['employee_name']=sm_ad5.sm_cost_center_name
sm_ad5['am_cost_center']=''
sm_ad5['am_cost_center_name']=''
sm_ad5['am_cost_center_full_name']=''
sm_ad5['record_date']=rec_date

sm_adopt=sm_ad5[['metric_name','level','acf2_id','employee_name','november','december','january',
        'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## make a copy for Total
sm_adopt1=sm_adopt.copy()
sm_adopt1['scorecard_filter']='Total'


In [ ]:
###################################################################################################
#################################### ROLL UP TO NATIONAL OFFICE ###################################
#### transpose data
a =adopt_full1.pivot_table(index=['metric_name'], 
                    columns=['month_dt'],
                     values='Branch Adoption percent', aggfunc='first', fill_value=0).reset_index()

## create a copy
adopt_full3=adopt_full1.copy()

## create a metric name
adopt_full3['metric_name']='Branch Adoption new accounts'

b =adopt_full3.pivot_table(index=['metric_name'], 
                    columns=['month_dt'],
                     values='bbs_count', aggfunc='first', fill_value=0).reset_index()

##### append the two dataframes (2 metrics)
adopt_all=pd.concat([a,b],axis=0)

cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in adopt_all]

adopt_all.reset_index(drop=False, inplace=True)
nso_ad= pd.concat([net_a[complementary], adopt_all], axis=1, join='outer',sort=False)

##delete the extra filler row
nso_ad= nso_ad.drop(labels='a', axis=0)

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in nso_ad]
nso_ad2= pd.concat([net_a[complementary], nso_ad], axis=1, join='outer',sort=False)

##delete the extra filler row
nso_ad2= nso_ad2.drop(labels='a', axis=0)
##convert to float
nso_ad2=nso_ad2.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

nso_ad2['q1'] = nso_ad2[['november','december','january']].mean(axis=1)
nso_ad2['q2'] = nso_ad2[['february','march','april']].mean(axis=1)
nso_ad2['q3'] = nso_ad2[['may','june','july']].mean(axis=1)
nso_ad2['q4'] = nso_ad2[['august','september','october']].mean(axis=1)
nso_ad2['ytd'] = nso_ad2[['november','december','january','february','march','april','may','june','july','august','september','october']].mean(axis=1)

## replace 0 with NaN so median only counts non-zeros
nso_ad2=nso_ad2.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_nso_medianq1=nso_ad2.groupby(['metric_name'], as_index=False)['q1'].median()
df_nso_medianq2=nso_ad2.groupby(['metric_name'], as_index=False)['q2'].median()
df_nso_medianq3=nso_ad2.groupby(['metric_name'], as_index=False)['q3'].median()
df_nso_medianq4=nso_ad2.groupby(['metric_name'], as_index=False)['q4'].median()
df_nso_medianytd=nso_ad2.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_nso_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_nso_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_nso_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_nso_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_nso_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
nso_ad1=pd.merge(pd.merge(nso_ad2,df_nso_medianq1,
                on=['metric_name'],
                how='left'),df_nso_medianq2,on=['metric_name'],how='left')

nso_ad0=pd.merge(pd.merge(nso_ad1,df_nso_medianq3,
                on=['metric_name'],
                how='left'),df_nso_medianq4,on=['metric_name'],how='left')

nso_ad3=pd.merge(nso_ad0,df_nso_medianytd,on=['metric_name'],how="left")

nso_ad3['acf2_id']=''
nso_ad3['level']='National Office'
nso_ad3['scorecard_filter']='Total'
nso_ad3['employee_name']='National Office'
nso_ad3['am_cost_center']=''
nso_ad3['am_cost_center_name']=''
nso_ad3['sm_cost_center']=''
nso_ad3['sm_cost_center_name']=''
nso_ad3['am_cost_center_full_name']=''
nso_ad3['record_date']=rec_date

nso_adopt=nso_ad3[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
######################## CONCATENATE ALL DATAFRAMES ##############################################
branch_adopt_all=pd.concat([adopt_am,adopt_am1,sm_adopt,sm_adopt1,nso_adopt],axis=0)


In [ ]:
branch_adopt_all.scorecard_filter.unique()
branch_adopt_all.groupby(['level', 'scorecard_filter'], as_index=False).first()

In [ ]:
branch_adopt_all.record_date.unique()

In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(branch_adopt_all).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_OUTPUT_BRANCH_ADOPTION") 


In [ ]:
## END OF PROGRAM